In [11]:
import pandas as pd
import os

# Suppress OpenGL warnings
os.environ["LIBGL_DEBUG"] = "quiet"
os.environ["MESA_LOADER_DRIVER_OVERRIDE"] = "swrast"  # Force software rendering (optional)

# Your imports and script logic here

# Load the Excel file
data_path = "/home/harshit/Desktop/hi/influencer_analysis/src/Assignment Data.xlsx"
df = pd.read_excel(data_path)


# Add columns to track processing status
df["HasFaces"] = False
df["FaceID"] = None
df["Processed"] = False
print(df.head())
import os
import sys

# Suppress OpenGL warnings
sys.stderr = open(os.devnull, "w")



   Performance                                          Video URL  HasFaces  \
0       1.1060  https://fgimagestorage.blob.core.windows.net/f...     False   
1       2.2447  https://fgimagestorage.blob.core.windows.net/f...     False   
2       2.0126  https://fgimagestorage.blob.core.windows.net/f...     False   
3       1.7708  https://fgimagestorage.blob.core.windows.net/f...     False   
4       0.6293  https://fgimagestorage.blob.core.windows.net/f...     False   

  FaceID  Processed  
0   None      False  
1   None      False  
2   None      False  
3   None      False  
4   None      False  


In [ ]:
import requests
import os

#function to download the video locally
def download_video(url, output_dir="./videos"):
    os.makedirs(output_dir, exist_ok=True)
    filename = url.split("/")[-1]  # Use the last part of the URL as the filename
    filepath = os.path.join(output_dir, filename)
    print("filenama",filename)
    filename = filepath.split("/")[-1]

    if os.path.exists(filepath):
        print(f"Video already exists: {filepath}")
        return filepath , filename
    
    response = requests.get(url, stream=True)


    if response.status_code == 200:
        with open(filepath, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
        print(f"Downloaded video: {filepath}")

        return filepath,filename
    else:
        print(f"Failed to download video from {url}")
        return None , None

In [ ]:
import cv2
import os

# extract mutiple frames per video for my device its every 30 frames it registers
def extract_frames(video_path, output_dir="./frames", frame_rate=1):
    # Use the video filename (without extension) to create a subfolder
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    video_frames_dir = os.path.join(output_dir, video_name)
    
    os.makedirs(video_frames_dir, exist_ok=True)  # Create a unique folder for the video's frames

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_interval = max(1, int(fps / frame_rate))  # Ensure the interval is at least 1
    success, frame = cap.read()
    count = 0

    while success:
        if count % frame_interval == 0:
            frame_filename = os.path.join(video_frames_dir, f"frame_{count}.jpg")
            cv2.imwrite(frame_filename, frame)
        success, frame = cap.read()
        count += 1
    cap.release()
    print(f"Frames extracted for {video_name} to: {video_frames_dir}")


In [ ]:
import cv2
import mediapipe as mp
import os
from collections import Counter
from deepface import DeepFace
import numpy as np

# Initialize Mediapipe face detection
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils


def are_faces_similar(embedding1, embedding2, threshold=0.9):
    """Check if two face embeddings are similar based on a threshold."""
    distance = np.linalg.norm(embedding1 - embedding2)  # Euclidean distance
    return distance < threshold

def detect_faces(frame_path):
    """Detect faces in a single frame and return cropped faces with bounding boxes."""
    image = cv2.imread(frame_path)
    if image is None:
        print(f"Error: Could not read image from {frame_path}")
        return [], image

    cropped_faces = []
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.9) as face_detection:
        results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.detections:
            for detection in results.detections:
                bbox = detection.location_data.relative_bounding_box
                h, w, _ = image.shape
                x_min = int(bbox.xmin * w)
                y_min = int(bbox.ymin * h)
                box_width = int(bbox.width * w)
                box_height = int(bbox.height * h)

                # Ensure the bounding box is within image bounds
                x_min = max(0, x_min)
                y_min = max(0, y_min)
                x_max = min(w, x_min + box_width)
                y_max = min(h, y_min + box_height)

                # Crop the detected face
                cropped_face = image[y_min:y_max, x_min:x_max]
                cropped_faces.append((cropped_face, (x_min, y_min, x_max, y_max)))

    return cropped_faces, image

def count_faces_in_video(frames_dir,fire_name):
    """Count faces across all frames in a directory, grouping similar faces."""
    frames_dir=frames_dir+"/"+fire_name
    print("this is me",frames_dir)
    if not os.path.exists(frames_dir):
        print(f"Error: Directory {frames_dir} does not exist.")
        return Counter(), {}

    face_counter = Counter()
    face_to_frame_map = {}
    known_embeddings = []  # Store unique face embeddings
    known_face_ids = []    # Map embeddings to unique face IDs

    for frame_file in os.listdir(frames_dir):
        frame_path = os.path.join(frames_dir, frame_file)
        if not frame_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue  # Skip non-image files

        faces, image = detect_faces(frame_path)
        if faces:
            for cropped_face, bbox in faces:
                try:
                    # Step 1: Extract face embedding
                    embedding = DeepFace.represent(img_path=cropped_face, model_name="VGG-Face")[0]["embedding"]

                    # Step 2: Check if this embedding matches any known embedding
                    match_found = False
                    for i, known_embedding in enumerate(known_embeddings):
                        if are_faces_similar(embedding, known_embedding):
                            face_id = known_face_ids[i]
                            face_counter[face_id] += 1
                            match_found = True
                            break

                    # Step 3: If no match found, assign a new ID
                    if not match_found:
                        face_id = len(known_embeddings) + 1  # Generate a new face ID
                        known_embeddings.append(embedding)
                        known_face_ids.append(face_id)
                        face_counter[face_id] += 1
                        # Store the first occurrence of this face for visualization
                        if face_id not in face_to_frame_map:
                            face_to_frame_map[face_id] = (image, (cropped_face, bbox))
                except Exception as e:
                    print(f"Error processing face: {e}")

    return face_counter, face_to_frame_map


In [ ]:
import numpy as np
import os
import cv2


def save_face_embedding(face_id, face_embedding, vidname, embedding_dir="./embeddings"):
    """
    Save the face embedding or unique identifier to a file.
    :param face_id: Unique ID of the face.
    :param face_embedding: Data to save (e.g., an array or identifier).
    :param vidname: Name of the video to organize embeddings.
    :param embedding_dir: Directory to save the embeddings.
    :return: Path to the saved embedding file.
    """
    # Create the directory if it doesn't exist
    folder_path = os.path.join(embedding_dir, vidname)
    os.makedirs(folder_path, exist_ok=True)  # Ensure the subdirectory is created

    # Save the embedding inside the subdirectory
    embedding_path = os.path.join(folder_path, f"face_{face_id}.npy")
    np.save(embedding_path, face_embedding)  # Save the face embedding as a .npy file
    print(f"Saved face embedding: {embedding_path}")
    return embedding_path



def display_and_save_face(image, videoid, face, output_dir="./faces"):
    """
    Handle the structure of `face` dynamically.
    Draw, display, and save a detected face.
    """
    # Check the structure of `face`
    if isinstance(face, tuple) and len(face) == 2:
        cropped_face, bbox = face
    elif isinstance(face, dict) and 'box' in face:
        bbox = face['box']
        cropped_face = image[bbox[1]:bbox[1] + bbox[3], bbox[0]:bbox[0] + bbox[2]]
    else:
        raise ValueError(f"Unexpected face structure: {face}")


    # Save the cropped face
    face_dir = os.path.join(output_dir, str(videoid))
    os.makedirs(face_dir, exist_ok=True)
    face_path = os.path.join(face_dir, "face.jpg")
    cv2.imwrite(face_path, cropped_face)
    print(f"Saved cropped face to: {face_path}")

#   # Optional: Display the face
#     cv2.imshow("Cropped Face", cropped_face)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()  

    return face_path




In [ ]:
from collections import Counter
import numpy as np


def process_video(row, video_dir="./videos", frames_dir="./frames", embedding_dir="./embeddings"):
    """Process a single video: download, detect faces, and save results."""
    url = row["Video URL"]
    performance = row["Performance"]

    # Step 1: Download video
    video_path,video_name = download_video(url, video_dir)

    # Step 2: Extract frames
    extract_frames(video_path, frames_dir)

    # Step 3: Detect faces and count occurrences
    face_counter, face_to_frame_map = count_faces_in_video(frames_dir,video_name)
    if face_counter:
        most_frequent_face_id = face_counter.most_common(1)[0][0]
        print(f"Most frequent face ID: {most_frequent_face_id} with {face_counter[most_frequent_face_id]} occurrences")
        # Display the most frequent face
        image, face = face_to_frame_map[most_frequent_face_id]
        display_and_save_face(image,video_name,face)
    else:
        print("No faces detected in the video.")

    if len(face_counter) == 0:
        return None, False  # No faces detected

    # Step 4: Identify most frequent face
    most_frequent_face = face_counter.most_common(1)[0]
    most_frequent_face_id = face_counter.most_common(1)[0][0]
    image, (cropped_face, bbox) = face_to_frame_map[most_frequent_face_id]

    try:
        # Step 5: Extract face embedding with enforce_detection=False
        embedding = DeepFace.represent(
            img_path=cropped_face, 
            model_name="VGG-Face", 
            enforce_detection=False
        )[0]["embedding"]

        # Step 6: Save face embedding
        save_face_embedding(most_frequent_face_id, embedding, row["Video URL"].split("/")[-1], embedding_dir)

        return most_frequent_face_id, True
    except ValueError as e:
        print(f"Error processing face in video {row['Video URL']}: {e}")
        return None, False


# Output directories
videos_dir = "./videos"
frames_dir = "./frames"
embeddings_dir = "./embeddings"

# Create directories
os.makedirs(videos_dir, exist_ok=True)
os.makedirs(frames_dir, exist_ok=True)
os.makedirs(embeddings_dir, exist_ok=True)


In [ ]:
# DataFrame to store videos without faces
no_faces_df = pd.DataFrame(columns=["Performance", "Video URL"])
faces_df=pd.DataFrame(columns=["Performance", "Video URL"])

# Iterate through the dataset
for index, row in df.iterrows():
    print(f"Processing row {index + 1}/{len(df)}: {row['Video URL']}")

    # Process the video
    face_id, has_faces = process_video(row, videos_dir, frames_dir, embeddings_dir)

    if has_faces:
        df.at[index, "HasFaces"] = True
        df.at[index, "FaceID"] = face_id
    else:
        no_faces_df = pd.concat([no_faces_df, pd.DataFrame([row])], ignore_index=True)

    # Mark as processed
    df.at[index, "Processed"] = True

# Create the complement DataFrame: rows in df but not in no_faces_df
faces_df = df[~df.index.isin(no_faces_df.index)].copy()

# Save results
df.to_excel("processed_with_faces_or_not.xlsx", index=False)
no_faces_df.to_excel("videos_without_faces.xlsx", index=False)
faces_df.to_excel("videos_with_faces.xlsx", index=False)

print("Processing complete!")



Processing row 1/268: https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-999607261342550
filenama hd-999607261342550
Video already exists: ./videos/hd-999607261342550
Frames extracted for hd-999607261342550 to: ./frames/hd-999607261342550
this is me ./frames/hd-999607261342550
Error processing face: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error processing face: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
No faces detected in the video.
Processing row 2/268: https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-997580728807604
filenama hd-997580728807604
Video already exists: ./videos/hd-997580728807604
Frames extracted for hd-997580728807604 to: ./frames/hd-997580728807604
this is me ./frames/hd-997580728807604
No faces detected in the video.
Processing row 3/268: 

In [30]:
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
from IPython.display import display, HTML

# Function to load embeddings and metadata
def load_embeddings(embedding_dir, performance_data):
    """
    Load all embeddings and their metadata from the directory, including performance points.
    :param embedding_dir: Path to the directory containing embeddings.
    :param performance_data: Dictionary mapping video IDs to performance points.
    :return: Tuple of embeddings and metadata.
    """
    embeddings = []
    metadata = []

    for video_id in os.listdir(embedding_dir):
        video_path = os.path.join(embedding_dir, video_id)
        if os.path.isdir(video_path):
            for file in os.listdir(video_path):
                if file.endswith(".npy"):
                    embedding_path = os.path.join(video_path, file)
                    embeddings.append(np.load(embedding_path))
                    metadata.append({
                        "video_id": video_id,
                        "face_file": embedding_path,
                        "face_id": f"{video_id}_Face_1",
                        "performance_points": performance_data.get(video_id, 0)  # Default to 0 if not found
                    })

    return np.array(embeddings), metadata

# Function to group similar faces based on embeddings
def group_similar_faces(embeddings, metadata, threshold=0.9):
    """Group similar faces based on cosine similarity."""
    groups = []
    used = set()

    for i, emb1 in enumerate(embeddings):
        if i in used:
            continue
        group = [metadata[i]]
        used.add(i)
        for j, emb2 in enumerate(embeddings):
            if j != i and j not in used:
                similarity = cosine_similarity(emb1.reshape(1, -1), emb2.reshape(1, -1))[0][0]
                if similarity >= threshold:
                    group.append(metadata[j])
                    used.add(j)
        groups.append(group)

    return groups

# Function to create a face table with images
def create_face_table_with_images(groups, faces_dir):
    """Create a DataFrame with face images, video URLs, and average performance points."""
    rows = []
    for group_id, group in enumerate(groups):
        video_urls = [f"https://fgimagestorage.blob.core.windows.net/facebook-assets/{item['video_id']}" for item in group]
        performance_points = sum(item["performance_points"] for item in group) / len(group)

        # Get face image path
        face_file_path = os.path.join(faces_dir, group[0]['video_id'], f"face.jpg")  # Assuming saved as 'face.jpg'
        if os.path.exists(face_file_path):
            face_image_path = face_file_path
        else:
            face_image_path = None  # Handle cases where the image doesn't exist

        rows.append({
            "Face Image Path": face_image_path,
            "Video URLs": ", ".join(video_urls),
            "Average Performance Points": performance_points
        })

    return pd.DataFrame(rows)

# Function to display the face table inline in Jupyter Notebook
def display_face_table_with_images(df):
    """Display a table in Jupyter Notebook with face images and other information."""
    html = """
    <table border="1">
        <tr>
            <th>Index</th>
            <th>Face Image</th>
            <th>Video URLs</th>
            <th>Average Performance Points</th>
        </tr>
    """
    for idx, row in df.iterrows():
        face_image_html = (
            f'<img src="{row["Face Image Path"]}" width="100">' if row["Face Image Path"] else "Image Not Found"
        )
        html += f"""
        <tr>
            <td>{idx + 1}</td>  <!-- Ensure sequential index -->
            <td>{face_image_html}</td>
            <td>{row["Video URLs"]}</td>
            <td>{row["Average Performance Points"]:.2f}</td>
        </tr>
        """
    html += "</table>"
    display(HTML(html))

# Main Execution

# Paths to data and directories
data_path = "/home/harshit/Desktop/hi/influencer_analysis/src/Assignment Data.xlsx"
embedding_dir = "./embeddings"
faces_dir = "./faces"

# Load the dataset
df = pd.read_excel(data_path)

# Load performance data
performance_data = {}
for _, row in df.iterrows():
    video_id = row["Video URL"].split("/")[-1]  # Extract the video ID from the URL
    performance_data[video_id] = row["Performance"]

# Load embeddings with metadata
embeddings, metadata = load_embeddings(embedding_dir, performance_data)

# Group similar faces
groups = group_similar_faces(embeddings, metadata, threshold=0.9)

# Create the face table
face_table = create_face_table_with_images(groups, faces_dir)

# Sort the face table by "Average Performance Points" in descending order
face_table = face_table.sort_values(by="Average Performance Points", ascending=False).reset_index(drop=True)  # Reset index

# Save the table to Excel
face_table.to_excel("face_video_mapping_sorted.xlsx", index=False)
print("Face-video mapping saved to 'face_video_mapping_sorted.xlsx'.")

# Optional: Display the table inline
display_face_table_with_images(face_table)


Face-video mapping saved to 'face_video_mapping_sorted.xlsx'.


In [ ]:
#lets do the faceless


In [ ]:
import os
import pandas as pd
from skimage.metrics import structural_similarity as ssim
import cv2
import numpy as np
from IPython.display import display, HTML

# Load the dataset
dataset_path = "/home/harshit/Desktop/hi/influencer_analysis/notebooks/videos_without_faces.xlsx"
df = pd.read_excel(dataset_path)

# Extract video IDs
df["Video ID"] = df["Video URL"].apply(lambda x: x.split("/")[-1])

# Function to load the last frame from the directory
def load_last_frame(video_id, frame_dir="/home/harshit/Desktop/hi/influencer_analysis/notebooks/frames", target_size=(300, 300)):
    video_frame_dir = os.path.join(frame_dir, video_id)
    if not os.path.exists(video_frame_dir):
        print(f"Frame directory not found for video ID: {video_id}")
        return None
    
    # Identify the last frame file based on the largest number in the filename
    frame_files = [f for f in os.listdir(video_frame_dir) if f.startswith("frame_") and f.endswith(".jpg")]
    if not frame_files:
        print(f"No frames found for video ID: {video_id}")
        return None

    # Extract frame number and find the highest
    frame_files_sorted = sorted(frame_files, key=lambda x: int(x.replace("frame_", "").replace(".jpg", "")))
    last_frame_path = os.path.join(video_frame_dir, frame_files_sorted[-1])  # Last frame in the sorted list
    frame = cv2.imread(last_frame_path, cv2.IMREAD_COLOR)
    frame_resized = cv2.resize(frame, target_size, interpolation=cv2.INTER_AREA)
    return frame_resized

# Function to compute similarity between frames
def compute_frame_similarity(frame1, frame2):
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    similarity, _ = ssim(gray1, gray2, full=True)
    return similarity

# Categorize videos based on last frames
def categorize_videos(df):
    video_frames = {}
    clusters = {}
    cluster_id = 1  # Start with 1 to match index format
    
    # Load all frames
    for video_id in df["Video ID"]:
        frame = load_last_frame(video_id)
        if frame is not None:
            video_frames[video_id] = frame
    
    # Compare frames and group similar ones
    for video_id, frame in video_frames.items():
        matched = False
        for cluster, cluster_frames in clusters.items():
            # Compare with a representative frame from the cluster
            similarity = compute_frame_similarity(frame, video_frames[cluster_frames[0]])
            if similarity > 0.9:  # Similarity threshold
                clusters[cluster].append(video_id)
                matched = True
                break
        if not matched:
            clusters[cluster_id] = [video_id]
            cluster_id += 1

    # Prepare results
    results = []
    for cluster, video_ids in clusters.items():
        avg_performance = df[df["Video ID"].isin(video_ids)]["Performance"].mean()
        video_urls = df[df["Video ID"].isin(video_ids)]["Video URL"].tolist()
        results.append({
            "Face Image": "Empty Profile Picture",
            "Video URLs": ", ".join(video_urls),
            "Average Performance Points": round(avg_performance, 2)
        })

    return results

# Display table in Jupyter Notebook
def display_face_table_with_images(df):
    """Display a table in Jupyter Notebook with face images and other information."""
    html = """
    <table border="1" style="border-collapse: collapse; width: 100%;">
        <tr style="background-color: #f2f2f2; text-align: left;">
            <th>Index</th>
            <th>Face Image</th>
            <th>Video URLs</th>
            <th>Average Performance Points</th>
        </tr>
    """
    for idx, row in df.iterrows():
        # Display the face image or placeholder
     
        html += f"""
        <tr>
            <td style="padding: 10px; text-align: center;">{idx+1}</td>
            <td style="padding: 10px; text-align: center;">influencer {idx+1}</td>
            <td style="padding: 10px; text-align: center;">no data available</td>
            <td style="padding: 10px;">{row["Video URLs"]}</td>
            <td style="padding: 10px; text-align: center;">{row["Average Performance Points"]:.2f}</td>
        </tr>
        """
    html += "</table>"
    display(HTML(html))

# Categorize videos
results = categorize_videos(df)

# Sort results by Average Performance Points in descending order
sorted_results = sorted(results, key=lambda x: x["Average Performance Points"], reverse=True)

# Save results to Excel
output_df = pd.DataFrame(sorted_results)
output_file = "faceless_categorized_videos_sorted.xlsx"
output_df.to_excel(output_file, index=False)

print(f"Results saved to {output_file}")

# Display results in Jupyter Notebook
output_df_for_display = pd.DataFrame(sorted_results)
display_face_table_with_images(output_df_for_display)


Results saved to categorized_videos_sorted.xlsx


/tmp/ipykernel_5827/1590775590.py:117: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  output_df.to_excel(output_file, index=False)


In [16]:
import pandas as pd
from IPython.core.display import display, HTML

# Load the face and faceless datasets
face_data = pd.read_excel("/home/harshit/Desktop/hi/influencer_analysis/notebooks/face_video_mapping_sorted.xlsx")
faceless_data = pd.read_excel("/home/harshit/Desktop/hi/influencer_analysis/notebooks/faceless_categorized_videos_sorted.xlsx")

# Combine both datasets
combined_data = pd.concat([face_data, faceless_data], ignore_index=True)

# Handle missing "Face Image Path" by replacing with the default image
default_image = "/home/harshit/Desktop/hi/influencer_analysis/notebooks/emptyprofile.jpg"
combined_data["Face Image Path"] = combined_data["Face Image Path"].fillna(default_image)

# Remove the "Face Image" column if it exists
if "Face Image" in combined_data.columns:
    combined_data = combined_data.drop(columns=["Face Image"])

# Sort by "Average Performance Points" in descending order
combined_data = combined_data.sort_values(by="Average Performance Points", ascending=False).reset_index(drop=True)

# Save the combined dataset to Excel
output_file = "combined_videos_sorted.xlsx"
combined_data.to_excel(output_file, index=False)
print(f"Combined dataset saved to {output_file}")

# Display the table using HTML
def display_combined_table_html(df):
    html = """
    <table border="1" style="border-collapse: collapse; width: 100%;">
        <tr>
            <th>Index</th>
            <th>Face Image</th>
            <th>Video URLs</th>
            <th>Average Performance Points</th>
        </tr>
    """
    for idx, row in df.iterrows():
        face_image_html = f'<img src="{row["Face Image Path"]}" width="100">'
        html += f"""
        <tr>
            <td>{idx + 1}</td>
            <td>{face_image_html}</td>
            <td>{row["Video URLs"]}</td>
            <td>{row["Average Performance Points"]:.2f}</td>
        </tr>
        """
    html += "</table>"
    display(HTML(html))

# Display the combined table
display_combined_table_html(combined_data)


Combined dataset saved to combined_videos_sorted.xlsx


/tmp/ipykernel_5827/287302417.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
/tmp/ipykernel_5827/287302417.py:24: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  combined_data.to_excel(output_file, index=False)


Index,Face Image,Video URLs,Average Performance Points
1,,https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-983335896508749,2.26
2,,https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-992418235673669,2.01
3,,"https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-986360206629277, https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-8796411607036992",1.80
4,,https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-803897110603133,1.63
5,,https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-604041931565137,1.53
6,,https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-671839840899900,1.47
7,,"https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-975523146942238, https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-932296771464171, https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-621652523105267, https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-904480964010630, https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-901272811190482, https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-874190910572544",1.38
8,,"https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-791572172961239, https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-803990118540094, https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-8008073409211713, https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-424802420355057",1.31
9,,"https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-929300948587514, https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-7995631707140182, https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-797179088937277",1.25
10,,https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-620702145697389,1.24


In [15]:
good_select=combined_data.iloc[0]
print(good_select)

Face Image Path                             ./faces/hd-983335896508749/face.jpg
Video URLs                    https://fgimagestorage.blob.core.windows.net/f...
Average Performance Points                                                2.263
Name: 0, dtype: object
